In [4]:
import sys
print(sys.version)

3.12.9 | packaged by conda-forge | (main, Feb 14 2025, 08:00:06) [GCC 13.3.0]


In [51]:
import uproot
import awkward as ak
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os #for looping over files in a directory
import math
import pandas as pd
from matplotlib.ticker import ScalarFormatter
import json
from pathlib import Path
import vector

In [6]:
dataset_runnable = json.loads(Path("af_v2_onefile.json").read_text())
print(dataset_runnable)
file_to_open=list(dataset_runnable['Znunugamma']['files'].keys())[0]
tree_name="analysis"

{'Znunugamma': {'files': {'/data/maclwong/Ben_Bkg_Samples/v2/user.bhodkins.700401.Znunugamma.mc20e.v2.0_ANALYSIS.root/user.bhodkins.42165174._000001.ANALYSIS.root': {'num_entries': 872532, 'object_path': 'analysis', 'steps': [[0, 51326], [51326, 102652], [102652, 153978], [153978, 205304], [205304, 256630], [256630, 307956], [307956, 359282], [359282, 410608], [410608, 461934], [461934, 513260], [513260, 564586], [564586, 615912], [615912, 667238], [667238, 718564], [718564, 769890], [769890, 821216], [821216, 872532]], 'uuid': '0cb0a828-a95a-11ef-b619-5c041cacbeef'}}, 'form': None, 'metadata': {'genFiltEff': 1.0, 'luminosity': 140, 'process': 'Znunugamma', 'xs': 56438.0}}}


In [7]:
data=None
with uproot.open(file_to_open) as f:
    #print("Keys in the ROOT file:", f.keys())
    #tree = f['analysis;1']
    #tree.show("name")

    tree_name = list(f.keys())[0]  # Get the first available tree
    tree = f[tree_name]  # Load the tree

    # Print only the variable names
    print("\n".join(tree.keys()))
    data=tree.arrays()

actualInteractionsPerCrossing
averageInteractionsPerCrossing
weight_beamspot
eventNumber
in_vgamma_overlap_7
in_vgamma_overlap_10
in_vgamma_overlap_15
in_vgamma_overlap_20
mcChannelNumber
runNumber
trigPassed_HLT_e120_lhloose
trigPassed_HLT_e140_lhloose_nod0
trigPassed_HLT_e24_lhmedium_L1EM20VH
trigPassed_HLT_e26_lhtight_nod0_ivarloose
trigPassed_HLT_e60_lhmedium
trigPassed_HLT_e60_lhmedium_nod0
trigPassed_HLT_mu20_iloose_L1MU15
trigPassed_HLT_mu26_ivarmedium
trigPassed_HLT_mu40
trigPassed_HLT_mu50
trigPassed_HLT_xe100_mht_L1XE50
trigPassed_HLT_xe110_mht_L1XE50
trigPassed_HLT_xe110_pufit_L1XE50
trigPassed_HLT_xe110_pufit_L1XE55
trigPassed_HLT_xe110_pufit_xe65_L1XE50
trigPassed_HLT_xe110_pufit_xe70_L1XE50
trigPassed_HLT_xe70_mht
trigPassed_HLT_xe90_mht_L1XE50
el_charge
el_eta
el_phi
el_truthOrigin
el_truthType
el_truthcharge
el_truthe
el_trutheta
el_truthpdgId
el_truthphi
el_truthpt
jet_eta
jet_btag_select
jet_phi
mu_charge
mu_eta
mu_phi
mu_truthOrigin
mu_truthType
mu_truthcharge
mu_tru

In [65]:
met_mask=(data.met_met_NOSYS<250000)
el_mask=(ak.count(data.el_pt_NOSYS>0,axis=1)==0)
mu_mask=(ak.count(data.mu_pt_NOSYS>0,axis=1)==0)
vgam_mask=(data.in_vgamma_overlap_7>0)
leadjet_mask=(ak.count(data.jet_pt_NOSYS>100000,axis=1)>0)
btag_mask=(ak.sum(data.jet_select_btag_NOSYS,axis=1)==0)
# mindphi requirement?
met=vector.zip({'pt': data.met_met_NOSYS, 'eta': np.zeros(len(data)), 'phi': data.met_phi_NOSYS, 'mass': np.zeros(len(data))})
print(met)
jets = vector.zip({'pt': data.jet_pt_NOSYS, 'eta': data.jet_eta, 'phi': data.jet_phi, 'mass': data.jet_m_NOSYS})
mindphi_mask=(ak.min(jets.deltaphi(met))>0.4)


presel_data=data[met_mask & el_mask & mu_mask & vgam_mask & leadjet_mask & btag_mask & mindphi_mask]
print(len(presel_data))

[{rho: 8.13e+05, phi: -0.353, eta: 0, tau: 0}, {...}, ..., {rho: 2.27e+05, ...}]
0


In [49]:
ph_preselection=ak.firsts((presel_data.ph_pt_NOSYS>10000) & 
                          ((abs(presel_data.ph_eta)<1.37) | ((abs(presel_data.ph_eta)>1.52) & 
                                                             (abs(presel_data.ph_eta)<2.37))) & 
                          ((presel_data.ph_isEM_NOSYS&0x45fc01)==0)
                         )
ph_presel_data=presel_data[ph_preselection]

In [50]:
print(len(ph_presel_data.ph_pt_NOSYS))
ph_tight_mask=(ak.firsts(ph_presel_data.ph_select_tightID_NOSYS==1))

200297


In [67]:
print(ak.firsts(photon_presel_data[photon_tight].ph_pt_NOSYS))

[2.2e+05, 7.03e+04, 2.57e+04, 3.58e+04, ..., 1.14e+05, 9.63e+04, 1.91e+05]
